In [45]:
import pandas as pd

In [46]:
file1 = pd.read_csv('C:\\Users\\kuzni\\PycharmProjects\\ScamDataCapture\\dfDATAfromFraudAdminBlocked7000.csv', encoding="utf-8-sig")
file1['label'] = 'spam'
file2 = pd.read_csv('C:\\Users\\kuzni\\PycharmProjects\\ScamDataCapture\\hams7000.csv', encoding="utf-8-sig")
file2['label'] = 'ham'
file = pd.merge(file1, file2, how='outer')
df = file
print(df['label'])

0          spam
1          spam
2          spam
3          spam
4          spam
           ... 
2263323     ham
2263324     ham
2263325     ham
2263326     ham
2263327     ham
Name: label, Length: 2263328, dtype: object
0          spam
1          spam
2          spam
3          spam
4          spam
           ... 
2263323     ham
2263324     ham
2263325     ham
2263326     ham
2263327     ham
Name: label, Length: 2263328, dtype: object


In [ ]:
for eachfile in range(len(df['bottom.payload'])):
    df.replace({r'[\n]+': ''}, regex=True, inplace=True)

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

In [ ]:
new = df["_id"].str.split(" ", n=5, expand=True)

In [ ]:
# making separate  column from new data frame
df["owner_Id"] = new[0]
df["ad_chat/ad_trigger"] = new[1]
df["chatId"] = new[2]
df["owner"] = new[3]
df["recipient"] = new[4]

# Dropping old Name columns
df.drop(columns=["_id"], inplace=True)
df.drop(columns=["owner"], inplace=True)
# df display
print(df)

df['recipient'] = df['recipient'].str.replace("}","")
df['chatId'] = df['chatId'].str.replace(",","")
df['ad_chat/ad_trigger'] = df['ad_chat/ad_trigger'].str.replace("{","")
df['ad_chat/ad_trigger'] = df['ad_chat/ad_trigger'].str.replace(",","")

df['ad_chat'] = 'O'
df['ad_trigger'] = 'O'
print(df.columns)
print(df['ad_chat/ad_trigger'].index)

for eachrow in range(len(df['ad_chat/ad_trigger'])):
    if df['ad_chat/ad_trigger'][eachrow] == 'ad_chat':
        df.loc[eachrow, 'ad_chat'] = 1
    else:
        df.loc[eachrow,'ad_chat'] = 0

for eachrow in range(len(df['ad_chat/ad_trigger'])):
    if df['ad_chat/ad_trigger'][eachrow] == 'ad_trigger':
        df.loc[eachrow,'ad_trigger'] = 1
    else:
        df.loc[eachrow,'ad_trigger'] = 0

df.drop(columns=["ad_chat/ad_trigger"], inplace=True)

In [ ]:

aggregation_dict = {
    'recipient': 'nunique',
    'chatId': 'nunique',
    'ad_chat': 'sum',
    'ad_trigger': 'sum',
    # 'updated': 'max',
    'bottom.payload': ['count', 'nunique'],
    'bottom.payload': lambda x: x.tolist(),  # Combine messages into a list
}
new_messages_df = df.groupby("owner_Id").agg(aggregation_dict)
most_frequent_labels = df.groupby("owner_Id")["label"].apply(lambda x: x.mode().iloc[0])

# Create a dictionary mapping owner_id to the most frequent label
owner_to_label_dict = most_frequent_labels.to_dict()
new_messages_df['owner_Id'] = new_messages_df.index
# Map the "owner_Id" to the most frequent label using the dictionary
new_messages_df['label'] = new_messages_df['owner_Id'].map(owner_to_label_dict)
new_messages_df['newest_message'] = df.groupby("owner_Id")["updated"].max()
new_messages_df['oldest_message'] = df.groupby("owner_Id")["updated"].min()
new_messages_df['total_messages'] = df.groupby("owner_Id")["bottom.payload"].count()
new_messages_df['total_unique_messages'] = df.groupby("owner_Id")["bottom.payload"].nunique()

new_messages_df.columns = ['total_recipients', 'total_chats', 'total_ad_chats', 'total_ad_triggers',
                           'text','owner_id','label','newest_message', 'oldest_message', 'total_messages', 'total_unique_messages']

print(new_messages_df)

In [ ]:
from datetime import datetime

for x in range(len(new_messages_df['newest_message'])):
     oid = new_messages_df.index[x]
     print(oid)
     start = new_messages_df.iloc[x]['oldest_message']
     # print(start)
     end = new_messages_df.iloc[x]['newest_message']
     # print(end)
     new_messages_df.loc[oid,'duration'] = datetime.strptime(str(end),"%Y-%m-%d %H:%M:%S")- datetime.strptime(str(start), "%Y-%m-%d %H:%M:%S")


print(new_messages_df)

In [ ]:
eachrow = 0
for eachrow in range(len(new_messages_df['duration'])):
     oid = new_messages_df.index[eachrow]
     if new_messages_df.iloc[eachrow]['duration']:
         head, sep, tail = str(new_messages_df.iloc[eachrow]['duration']).partition(' ')
         if head == 0:
             new_messages_df.loc[oid, 'chats_per_days'] = new_messages_df.loc[oid, 'total_chats']/0.001
         else:
             new_messages_df.loc[oid, 'chats_per_days'] = new_messages_df.loc[oid, 'total_chats']/float(head)
     else:
         new_messages_df.loc[oid, 'chats_per_days'] = 0.0001
     eachrow+=1

for eachrow in range(len(new_messages_df['duration'])):
     oid = new_messages_df.index[eachrow]
     if new_messages_df.iloc[eachrow]['duration']:
         head, sep, tail = str(new_messages_df.iloc[eachrow]['duration']).partition(' ')
         if head == 0:
             new_messages_df.loc[oid, 'recipients_per_days'] = new_messages_df.loc[oid, 'total_chats']/0.001
         else:
             new_messages_df.loc[oid, 'recipients_per_days'] = new_messages_df.loc[oid, 'total_recipients']/float(head)
     else:
         new_messages_df.loc[oid, 'recipients_per_days'] = 0.0001

print(new_messages_df[['text','duration']])

In [ ]:
arrayTest = []
for res in range(len(new_messages_df['text'])):
    test = new_messages_df.iloc[res]['text']
    arrayTest.append(''.join([str(ele) + " " for ele in test]))
print(arrayTest)

import re

# Clean the array
cleaned_array = []
for item in arrayTest:
    cleaned_item = re.sub(r'[\n\d📣☘️\\]', '', item)
    cleaned_array.append(cleaned_item.strip())

print(cleaned_array)

In [ ]:
import re
phrases_to_delete = ["посмотрел(а) ваш номер телефона","актуально","просмотрел(а) ваш номер телефона", "ваше объявление","Name:","bottom.payload","Name: , dtype: object","Name: , Length: , dtype: object","Name: , dtype: object",  "добавил(а) в Избранное", "dtype:" ,"object", "dtype: object", "просмотрел(а) ваш номер телефона", "Length: , dtype: object", "Ещё актуально?", "n/"]

# Construct a regular expression pattern with all the phrases to delete
pattern = r'\b(?:{})\b'.format('|'.join(map(re.escape, phrases_to_delete)))

# Clean the array by removing the specified phrases
cleaned_arr = [re.sub(pattern, '', item) for item in cleaned_array]

import langdetect
import spacy

def preprocess_text(text):
    # Check if input is a list of texts
    if isinstance(text, list):
        preprocessed_texts = []
        for t in text:
            preprocessed_texts.append(preprocess_single_text(t))
        return preprocessed_texts
    else:
        return preprocess_single_text(text)

def preprocess_single_text(text):
    # Detect language
    try:
        language = langdetect.detect(text)
    except langdetect.lang_detect_exception.LangDetectException:
        language = 'unknown'

    # Load the appropriate Spacy model and stop words for the detected language
    if language == 'hy':  # Armenian
        nlp = spacy.load('hy_core_news_sm')
    elif language == 'ru':  # Russian
        nlp = spacy.load('ru_core_news_sm')
    elif language == 'pl':  # Polish
        nlp = spacy.load('pl_core_news_sm')
    else:  # Default to English model
        nlp = spacy.load('en_core_web_sm')

    # Remove stop words and perform lemmatization after lowercasing
    doc = nlp(text.lower())
    lemmatized_words = [token.lemma_ for token in doc if not token.is_stop]

    return ' '.join(lemmatized_words)

cleaned_data = [re.sub(r'\s+', ' ', item) for item in preprocess_text(cleaned_arr)]
new_messages_df['text'] = cleaned_data

print(new_messages_df['text'])

In [ ]:
import os

def new_messages_to_file(df):
    # Create the directory
    directory = 'files'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the CSV files in the created directory
    for name in '1':
        df.to_csv(os.path.join(directory, f'{name}preprocessed_data_all.csv'), encoding="utf-8-sig")


new_messages_to_file(new_messages_df)